In [1]:
import cudaq
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [2]:
# ----------------------------
# Quantum kernel for Q-LR
# ----------------------------
@cudaq.kernel
def qlr_kernel(x: list[float], weights: list[float]):
    q = cudaq.qvector(len(x))
    # Encode features into rotations
    for i in range(len(x)):
        ry(x[i], q[i])
    # Linear ansatz (like weighted rotations)
    for i in range(len(x)):
        rz(weights[i] * x[i], q[i])
    # Entanglement
    for i in range(len(x) - 1):
        cx(q[i], q[i + 1])

def get_observable(n_qubits):
    return sum([cudaq.spin.z(i) for i in range(n_qubits)])

# ----------------------------
# Quantum Expectation + Grad
# ----------------------------
def quantum_expectation(x, weights, n_qubits):
    return cudaq.observe(qlr_kernel, get_observable(n_qubits), list(x[:n_qubits]), list(weights)).expectation()

def parameter_shift_gradient(x, weights, n_qubits, shift=np.pi/2):
    grads = np.zeros_like(weights)
    for i in range(len(weights)):
        shifted = weights.copy()

        shifted[i] += shift
        forward = quantum_expectation(x, shifted, n_qubits)

        shifted[i] -= 2 * shift
        backward = quantum_expectation(x, shifted, n_qubits)

        grads[i] = 0.5 * (forward - backward)
    return grads

# ----------------------------
# Torch Autograd Wrapper
# ----------------------------
class QLRFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, weights):
        x_np = x.detach().cpu().numpy()
        w_np = weights.detach().cpu().numpy()
        n_qubits = len(w_np)

        out_vals = [quantum_expectation(row, w_np, n_qubits) for row in x_np]
        ctx.save_for_backward(x, weights)
        return torch.tensor(out_vals, dtype=torch.float32, device=x.device).view(-1, 1)

    @staticmethod
    def backward(ctx, grad_output):
        x, weights = ctx.saved_tensors
        x_np, w_np = x.detach().cpu().numpy(), weights.detach().cpu().numpy()
        n_qubits = len(w_np)

        grad_w = np.zeros_like(w_np)
        for row in x_np:
            grad_w += parameter_shift_gradient(row, w_np, n_qubits)
        grad_w /= len(x_np)

        return None, torch.tensor(grad_w, dtype=torch.float32, device=weights.device)

# ----------------------------
# Q-LR Model
# ----------------------------
class QLR(nn.Module):
    def __init__(self, n_qubits=4):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(n_qubits))

    def forward(self, x):
        return QLRFunction.apply(x, self.weights)


In [3]:
# ----------------------------
# Load dataset
# ----------------------------
df = pd.read_csv("../../data/data.csv")
num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].mean())

drop_cols = ["ID", "Unnamed: 0", "e_Sint"]
features = [c for c in df.columns if c not in drop_cols and c != "Sint"]

X = df[features].values.astype(np.float32)
y = df["Sint"].values.astype(np.float32).reshape(-1, 1)

scalerX, scalery = StandardScaler(), StandardScaler()
X = scalerX.fit_transform(X)
y = scalery.fit_transform(y)

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ----------------------------
# Train Q-LR
# ----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = QLR(n_qubits=4).to(device)
opt = optim.Adam(model.parameters(), lr=0.1)
loss_fn = nn.MSELoss()

for epoch in range(5):
    model.train()
    opt.zero_grad()
    preds = model(X_train.to(device))
    loss = loss_fn(preds, y_train.to(device))
    loss.backward()
    opt.step()
    print(f"Epoch {epoch+1}, Loss={loss.item():.6f}")

Epoch 1, Loss=4.375508
Epoch 2, Loss=4.375508
Epoch 3, Loss=4.375508
Epoch 4, Loss=4.375508
Epoch 5, Loss=4.375508


In [4]:
# ----------------------------
# Evaluate
# ----------------------------
model.eval()
with torch.no_grad():
    y_pred = model(X_test.to(device)).cpu().numpy()
    y_true = y_test.cpu().numpy()

mse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
r2  = r2_score(y_true, y_pred)

print("\n--- Q-LR Metrics (Sint) ---")
print("MSE :", mse)
print("MAE :", mae)
print("R²  :", r2)


--- Q-LR Metrics (Sint) ---
MSE : 4.864708423614502
MAE : 1.67332923412323
R²  : -3.653843879699707
